# Identify for each LED project/proposal how was the project's annual lighting cost calculated:

1)(maybe = proposed_wattage * proposed_operating_hours * proposed_technology_cost_per_watt_hour)

2)(total-wattage ÷ 1000) x  hours-per-day-used x 365 x your-kwh-cost = Annual cost for your system/project/proposal.


Tables and respective columns used:
1) Proposals - id (proposal id), project_id, name, active

2) Proposal_options - proposal_id, option_id

3) rate_schedules - name, kwh_cost, kwh_cost_simple, kw_demand_cost, demand_utilisation, rate_type

4) option_calculations - id (option_calc_id), 'proposed_kw', 'proposed_kwh', 'kw_proposed_annual_savings', 'replacement_cost_per_year', 'replacement_demand_cost_per_year', 'replacement_energy_cost_per_year', 'replacement_maintenance_cost_per_year', 'replacement_heating_cost_per_year', 'replacement_cooling_cost_per_year', 'total_fixture_cost', 'total_replacement_cost', 'total_savings_per_year', 'total_encentiv_savings', 'total_labor_cost', 'total_shipping_cost', 'total_misc_cost', 'total_tax_cost'

5) option_fixtures - product id, product name, product type, watt_hours_consumed, total_cost_per_year, 'item_watts', 'item_watt_hours_consumed', total_replacement_cost


### Which tables can be further used?
1) Option_calc_lightings - Proposed_quantity (But all/most values are 0)

2) Solutions - For solution_name, product_count

3) line_items 

In [8]:
import pyspark 
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Omdena").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [170]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

path = "Selected/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['option_calculations', 'option_calc_fixtures', 'option_calc_lightings', 'proposals', 'proposal_options', 'rate_schedules', 'solutions']


In [93]:
#Using Proposals Tables
step1 = proposals.select(['id','project_id','name','active']).orderBy("id").filter(proposals.active == True)
step1 = step1.withColumnRenamed('id','ID_proposal')
step1 = step1.withColumnRenamed('name', 'proposal_name')
step1.limit(10).toPandas()
step1.count()

8943

In [91]:
#Using Proposals_options Tables --------- for option_id
step2 = step1.join(proposal_options, step1.ID_proposal == proposal_options.proposal_id, how='left').select(['proposal_id','project_id', 'proposal_name', 'option_id']).filter(proposal_options.active == True)
step2.limit(20).toPandas()
step2.count()

8905

In [142]:
#Using rate_schedules ------- For Energy provider name, kwh_cost, demand_utilisation, rate_type
step3 = step2.join(rate_schedules, step2.project_id == rate_schedules.project_id, how='left') \
.select(['proposal_id',step2.project_id, 'proposal_name', 'option_id', 'name', 'kwh_cost', 'demand_utilization', 'rate_type']).filter(rate_schedules.active == True)
step3 = step3.withColumnRenamed('name','energy_provider_name')
step3.limit(20).toPandas()
step3.count()

8767

In [143]:
#Using option_calculations 
step4 = step3.join(option_calculations, step3.option_id == option_calculations.option_id, how='left')
step4 = step4.select(['proposal_id','proposal_name', step3.project_id, step3.option_id, 'id','energy_provider_name', 'kwh_cost', 'demand_utilization', 'rate_type', 'proposed_kw', 'proposed_kwh', \
                      'kw_proposed_annual_savings', 'replacement_cost_per_year', 'replacement_demand_cost_per_year', 'replacement_energy_cost_per_year', 'replacement_maintenance_cost_per_year', 'replacement_heating_cost_per_year', 'replacement_cooling_cost_per_year',\
                      'total_fixture_cost', 'total_replacement_cost', 'total_savings_per_year', 'total_encentiv_savings', 'total_labor_cost', 'total_shipping_cost', 'total_misc_cost', 'total_tax_cost'])
step4 = step4.withColumnRenamed('id','option_calc_id')
step4.limit(20).toPandas()
step4.count()

8767

In [165]:
#Using option_calc_fixtures -----watt_hours_consumed, total_cost_per_year, 'item_watts', 'item_watt_hours_consumed', product id, product name, product type
step5 = step4.join(option_calc_fixtures, step4.option_calc_id == option_calc_fixtures.option_calculation_id, how='left')
step5 = step5.select(['proposal_id','proposal_name', 'project_id', 'option_id', 'option_calc_id', 'product_id', 'product_type','product_name', \
                      'energy_provider_name', 'demand_utilization', 'rate_type', 'kwh_cost', 'item_watts', 'item_watt_hours_consumed', 'watts_per_product','proposed_kw', 'proposed_kwh', 'kw_proposed_annual_savings', 'energy_charge', 'total_cost_per_year', 'total_savings',\
                      option_calc_fixtures.total_replacement_cost,'replacement_cost_per_year', 'replacement_demand_cost_per_year', 'replacement_energy_cost_per_year', 'replacement_maintenance_cost_per_year', 'replacement_heating_cost_per_year', 'replacement_cooling_cost_per_year', \
                      'total_fixture_cost', 'total_savings_per_year', 'total_encentiv_savings', 'total_labor_cost', 'total_shipping_cost', 'total_misc_cost', 'total_tax_cost']).filter(option_calc_fixtures.active == True)
step5 = step5.na.drop("any") # dropping any row with missing values
step5.count()
step5.limit(20).toPandas()

,proposal_id,proposal_name,project_id,option_id,option_calc_id,product_id,product_type,product_name,energy_provider_name,demand_utilization,rate_type,kwh_cost,item_watts,item_watt_hours_consumed,watts_per_product,proposed_kw,proposed_kwh,kw_proposed_annual_savings,energy_charge,total_cost_per_year,total_savings,total_replacement_cost,replacement_cost_per_year,replacement_demand_cost_per_year,replacement_energy_cost_per_year,replacement_maintenance_cost_per_year,replacement_heating_cost_per_year,replacement_cooling_cost_per_year,total_fixture_cost,total_savings_per_year,total_encentiv_savings,total_labor_cost,total_shipping_cost,total_misc_cost,total_tax_cost
0,269,Big Grill & Tap,86,267,148,13114.0,Product,T8 LED U-Bent Lamp - Line Driven (MOQ: 6),Upper Marlboro,100,blended,0.0809,60.0,187200.0,15.0,23.385,72961.2,41.209,15.144480,-300.617681,-551.132415,198.4,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
1,269,Big Grill & Tap,86,267,148,13487.0,Product,7A19DLED30,Upper Marlboro,100,blended,0.0809,28.0,87360.0,7.0,23.385,72961.2,41.209,7.067424,-140.288251,-1062.182473,52.0,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
2,269,Big Grill & Tap,86,267,148,13114.0,Product,T8 LED U-Bent Lamp - Line Driven (MOQ: 6),Upper Marlboro,100,blended,0.0809,1260.0,3931200.0,15.0,23.385,72961.2,41.209,318.034080,-6312.971303,-11573.780722,4166.4,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
3,269,Big Grill & Tap,86,267,148,13114.0,Product,T8 LED U-Bent Lamp - Line Driven (MOQ: 6),Upper Marlboro,100,blended,0.0809,360.0,1123200.0,15.0,23.385,72961.2,41.209,90.866880,-1803.706087,-3306.794492,1190.4,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
4,269,Big Grill & Tap,86,267,148,11958.0,Product,200D 15W 3500K F,Upper Marlboro,100,blended,0.0809,60.0,187200.0,15.0,23.385,72961.2,41.209,15.144480,-300.617681,-420.864754,216.0,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
5,269,Big Grill & Tap,86,267,148,13114.0,Product,T8 LED U-Bent Lamp - Line Driven (MOQ: 6),Upper Marlboro,100,blended,0.0809,180.0,561600.0,15.0,23.385,72961.2,41.209,45.433440,-901.853043,-1653.397246,595.2,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
6,269,Big Grill & Tap,86,267,148,13321.0,Product,10P30DLED27FL,Upper Marlboro,100,blended,0.0809,60.0,187200.0,10.0,23.385,72961.2,41.209,15.144480,-300.617681,-2404.941449,192.0,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
7,269,Big Grill & Tap,86,267,148,13114.0,Product,T8 LED U-Bent Lamp - Line Driven (MOQ: 6),Upper Marlboro,100,blended,0.0809,120.0,374400.0,15.0,23.385,72961.2,41.209,30.288960,-601.235362,-1102.264831,396.8,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
8,269,Big Grill & Tap,86,267,148,11958.0,Product,200D 15W 3500K F,Upper Marlboro,100,blended,0.0809,420.0,1310400.0,15.0,23.385,72961.2,41.209,106.011360,-2104.323768,-2946.053275,1512.0,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0
9,269,Big Grill & Tap,86,267,148,11958.0,Product,200D 15W 3500K F,Upper Marlboro,100,blended,0.0809,60.0,187200.0,15.0,23.385,72961.2,41.209,15.144480,-300.617681,-561.153005,216.0,-117165.741204,0.0,5902.56108,0.0,-124506.828576,1438.526292,117055.72,-206469.233666,0.0,0.0,0.0,0.0,0.0


In [164]:
#writing to a csv file
step5.toPandas().to_csv("proposed_replacement_lighting_calculations_and_costs.csv")